# intermax os, stat, wait table을 전처리한 데이터에 대해 **RNN** 모델을 만들자

by pdm

In [3]:
# ----------------------------------------
# Library used for command parsing (maybe)
# ----------------------------------------
import argparse
import os
from tensorflow.contrib.learn.python.learn.utils import (
    saved_model_export_utils)
from tensorflow.contrib.training.python.training import hparam

# ---------------------------------------------------
# Library used for loading a file from Google Storage
# ---------------------------------------------------
from tensorflow.python.lib.io import file_io

# ---------------------------------------------------
# Library used for uploading a file to Google Storage
# ---------------------------------------------------
from googleapiclient import discovery
from oauth2client.client import GoogleCredentials

In [4]:
import tensorflow as tf
import numpy as np
import matplotlib
import os
import matplotlib.pyplot as plt
import csv

In [2]:
def MinMaxScaler(data):
    ''' Min Max Normalization
    Parameters, Returns
    ----------
    data : numpy.ndarray
        input data to be normalized
        shape: [Batch size, dimension]
    '''
    numerator = data - np.min(data, 0)
    denominator = np.max(data, 0) - np.min(data, 0)
    
    # noise term prevents the zero division
    return numerator / (denominator + 1e-7)

In [6]:
def load_series(filename):
  filename = filename[0] #filename : route (--train-files <route>)
  try:
    with file_io.FileIO(filename, mode='r') as csvfile:
      csvreader = csv.reader(csvfile)
      return csvreader
  except IOError:
    return None

In [7]:
def run_experiements(hparams):
  
  data = load_series(hparams.train_files)
  
  # train Parameters
  seq_length = 10
  data_dim = 53
  hidden_dim = 10 #??
  output_dim = 1
  learning_rate = 0.01
  iterations = 100

# ml-engine돌릴때 argument parsing을 위해 필요한 코드

In [8]:
if __name__ == '__main__':

	# ---------------------------------------------
	# command parsing from Google ML Engine Example
	# ---------------------------------------------
	parser = argparse.ArgumentParser()
	# Input Arguments
	parser.add_argument(
	  '--train-files',
	  help='GCS or local paths to training data',
	  nargs='+',
	  required=True
	)
	parser.add_argument(
	  '--num-epochs',
	  help="""\
	  Maximum number of training data epochs on which to train.
	  If both --max-steps and --num-epochs are specified,
	  the training job will run for --max-steps or --num-epochs,
	  whichever occurs first. If unspecified will run for --max-steps.\
	  """,
	  type=int,
	)
	parser.add_argument(
	  '--train-batch-size',
	  help='Batch size for training steps',
	  type=int,
	  default=40
	)
	parser.add_argument(
	  '--eval-batch-size',
	  help='Batch size for evaluation steps',
	  type=int,
	  default=40
	)

	# -------------------------------
	# If evaluation file is prepared,
	# change 'required' value
	# -------------------------------
	parser.add_argument(
	  '--eval-files',
	  help='GCS or local paths to evaluation data',
	  nargs='+',
	  required=False
	)
	# Training arguments
	parser.add_argument(
	  '--embedding-size',
	  help='Number of embedding dimensions for categorical columns',
	  default=8,
	  type=int
	)
	parser.add_argument(
	  '--first-layer-size',
	  help='Number of nodes in the first layer of the DNN',
	  default=100,
	  type=int
	)
	parser.add_argument(
	  '--num-layers',
	  help='Number of layers in the DNN',
	  default=4,
	  type=int
	)
	parser.add_argument(
	  '--scale-factor',
	  help='How quickly should the size of the layers in the DNN decay',
	  default=0.7,
	  type=float
	)
	parser.add_argument(
	  '--job-dir',
	  help='GCS location to write checkpoints and export models',
	  required=True
	)

	# Argument to turn on all logging
	parser.add_argument(
	  '--verbosity',
	  choices=[
	      'DEBUG',
	      'ERROR',
	      'FATAL',
	      'INFO',
	      'WARN'
	  ],
	  default='INFO',
	)
	# Experiment arguments
	parser.add_argument(
	  '--train-steps',
	  help="""\
	  Steps to run the training job for. If --num-epochs is not specified,
	  this must be. Otherwise the training job will run indefinitely.\
	  """,
	  type=int
	)
	parser.add_argument(
	  '--eval-steps',
	  help='Number of steps to run evalution for at each checkpoint',
	  default=100,
	  type=int
	)
	parser.add_argument(
	  '--export-format',
	  help='The input format of the exported SavedModel binary',
	  choices=['JSON', 'CSV', 'EXAMPLE'],
	  default='JSON'
	)

	args = parser.parse_args()

	# Set python level verbosity
	tf.logging.set_verbosity(args.verbosity)
	# Set C++ Graph Execution level verbosity
	os.environ['TF_CPP_MIN_LOG_LEVEL'] = str(
	  tf.logging.__dict__[args.verbosity] / 10)

	# Run the training job
	hparams=hparam.HParams(**args.__dict__)
	run_experiment(hparams)

usage: __main__.py [-h] --train-files TRAIN_FILES [TRAIN_FILES ...]
                   [--num-epochs NUM_EPOCHS]
                   [--train-batch-size TRAIN_BATCH_SIZE]
                   [--eval-batch-size EVAL_BATCH_SIZE]
                   [--eval-files EVAL_FILES [EVAL_FILES ...]]
                   [--embedding-size EMBEDDING_SIZE]
                   [--first-layer-size FIRST_LAYER_SIZE]
                   [--num-layers NUM_LAYERS] [--scale-factor SCALE_FACTOR]
                   --job-dir JOB_DIR
                   [--verbosity {DEBUG,ERROR,FATAL,INFO,WARN}]
                   [--train-steps TRAIN_STEPS] [--eval-steps EVAL_STEPS]
                   [--export-format {JSON,CSV,EXAMPLE}]
__main__.py: error: argument --train-files is required


SystemExit: 2

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
